# Diagnostics: mean bias and plotting of difference maps

A notebook to demonstrate the usage of an implemented plotting function in Valenspy

In [1]:
import valenspy as vp
from valenspy._utilities import load_yml
from pathlib import Path

from dask.diagnostics import ProgressBar
from valenspy.preprocessing_tasks.regrid import remap_cdo
import warnings
import xarray as xr

# Turn off all warnings
warnings.filterwarnings('ignore')


# define machine name - used for paths of (observational) datasets
machine = 'hortense'



## 0. Settings

In [2]:
model = 'CCLM'
ref_dataset = 'ERA5'
variable = 'tas'

## 1. Loading data

### 1.1 Model data

To be replaced with automatic input manager returning the ds

In [3]:
# get lookup file for model
mod_LOOKUP = load_yml(model+"_lookup")
mod_var = mod_LOOKUP[variable]['mod_name']

experiment = "EUR11_CO_TA_GC_TSO"
postproc_base_dir = "/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/postprocessing/"

# define the path
directory = Path(postproc_base_dir + experiment +'/'+mod_var + '/')

# define the CCLM files for the corresponding variable
mod_files = list(directory.glob(mod_var+"_daymean.nc")) # Select all the netCDF files in the directory
    
ds_mod_raw = xr.open_mfdataset(mod_files, combine="by_coords", chunks="auto")

### 1.2. Load reference data

here we will use ERA5

In [4]:
manager = vp.InputManager(machine=machine)

ds_obs = manager.load_data(ref_dataset,variable, period=[1995,1995],freq="hourly",region="europe")


File paths found:
/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/2m_temperature/era5-hourly-europe-2m_temperature-1995.nc
The file is ValEnsPy CF compliant.
100.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['tas']


## 2. preprocessing before calculating the diagnostic
in this example, this means selecting time and remapping

In [5]:
# regrid
# retrieve ERA5 gridfile - for regridding 
gridfile = manager._get_file_paths(ref_dataset,variable, period=[1995,1995],freq="hourly",region="europe")[0]
ds_mod = remap_cdo(gridfile, ds_mod_raw, remap_method = "con")

# resample ERA5 hourly data to daily
ds_obs = ds_obs.resample(time='1D').mean()    

# for both datasets, select summer months (JJA)
ds_mod = ds_mod.sel(time=ds_mod.time.dt.month.isin([6,7,8]))
ds_obs = ds_obs.sel(time=ds_obs.time.dt.month.isin([6,7,8]))


Now, both datasets are ready to be compared. 

## 3. Calculate the diagnostic and do plotting

In [ ]:
from valenspy.diagnostic